In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from rdkit import Chem
import numpy as np
import pandas as pd
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch.nn import BatchNorm1d
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_add_pool
from torch_geometric.data import DataLoader, Data
import copy
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from tqdm.notebook import tqdm

In [ ]:
paser = argparse.ArgumentParser()
args = paser.parse_args("")
args.seed = 123

np.random.seed(args.seed)
torch.manual_seed(args.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def features_to_id(features, intervals):
    """Convert list of features into index using spacings provided in intervals"""
    id = 0
    for k in range(len(intervals)):
        id += features[k] * intervals[k]
    id = id + 1
    return id

def atom_to_id(atom):
    """Return a unique id corresponding to the atom type"""
    features = get_feature_list(atom)
    return features_to_id(features, intervals)

def atom_features(atom):
    results = np.array(one_of_k_encoding_unk(atom.GetSymbol(),['C','N','O','S','F','Si','P','Cl','Br','Mg',
                                                               'Na','Ca','Fe','As','Al','I','B','V','K','Tl',
                                                               'Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn','H',  # H?
                                                               'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr',
                                                               'Pt','Hg','Pb','Unknown']) + 
                       one_of_k_encoding(atom.GetDegree(),[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) + 
                       one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6]) + 
                       [atom.GetFormalCharge(), atom.GetNumRadicalElectrons()] + 
                       one_of_k_encoding_unk(atom.GetHybridization(), [Chem.rdchem.HybridizationType.SP, 
                                                                       Chem.rdchem.HybridizationType.SP2, 
                                                                       Chem.rdchem.HybridizationType.SP3, 
                                                                       Chem.rdchem.HybridizationType.SP3D, 
                                                                       Chem.rdchem.HybridizationType.SP3D2]) + 
                       [atom.GetIsAromatic()])

    return np.array(results)

def get_bond_pair(mol):
    bonds = mol.GetBonds()
    res = [[],[]]
    for bond in bonds:
        res[0] += [bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()]
        res[1] += [bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()]
    return res

def mol2vec(mol):
    atoms = mol.GetAtoms()
    bonds = mol.GetBonds()
    node_f= [atom_features(atom) for atom in atoms]
    edge_index = get_bond_pair(mol)
    data = Data(x=torch.tensor(node_f, dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype=torch.long))
    return data

In [ ]:
def df_check(df):
    df['mol2vec'] = 'Yes'
    for i in range(df.shape[0]):
        try:
            m = Chem.MolFromSmiles(df['SMILES'].iloc[i])
            vec = mol2vec(m)
    #         print(i, m, vec)
        except:
            df['mol2vec'].iloc[i] = 'No'
    #         print('No')
            continue

    df = df[df['mol2vec'] != 'No'].sample(frac=1).reset_index(drop=True)
    del df['mol2vec']
    return df

In [ ]:
def make_mol(df):
    mols = {}
    for i in range(df.shape[0]):
        mols[Chem.MolFromSmiles(df['SMILES'].iloc[i])] = df['Class'].iloc[i]
    return mols

In [ ]:
def make_vec(mols):
    X = [mol2vec(m) for m in mols.keys()]
    for i, data in enumerate(X):
        y = list(mols.values())[i]
        data.y = torch.tensor([y], dtype=torch.long)
    return X

In [ ]:
def SMILES_to_InChIKey(df, col, i):
    try:
        s = df[col].iloc[i]
        m = Chem.MolFromSmiles(s)
        inc = Chem.inchi.MolToInchi(m)
        key = Chem.inchi.InchiToInchiKey(inc)
    except:
        key = np.nan
    return key

In [ ]:
df = pd.read_csv('data/All-Public dataset.csv', low_memory=False)
df = pd.concat([df['SMILES'], df['Class']], axis=1)
df

In [ ]:
def save_checkpoint(epoch, model, optimizer, filename):
    state = {
        'Epoch': epoch,
        'State_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, filename)

def train(model, device, optimizer, train_loader, criterion, args):
    train_correct = 0
    train_total = 0
    epoch_train_loss = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        labels = data.y.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        outputs.require_grad = False
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    epoch_train_loss /= len(train_loader)
    train_acc =  100 * train_correct / train_total
#     print('- Loss : %.4f' % epoch_train_loss)
#     print('- Accuracy : %.4f' % train_acc)
    return model, epoch_train_loss, train_acc


def test(model, device, test_loader, args):
    model.eval()
    classes = ('RB', 'NRB')
    class_correct = list(0. for i in range(len(classes)))
    class_total = list(0. for i in range(len(classes)))
    correct = 0
    total = 0
    nb_classes = len(classes)
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    test_hist = {"test_acc":[]}
    y_score =[]
    y_test =[]
    data_total = []
    pred_data_total = []
    roc_total = dict()
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device)
            labels = data.y.to(device)
            data_total += data.y.tolist()
            outputs = model(data)
            pred_data_total += outputs.view(-1).tolist()
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            y_score.append(outputs.cpu().numpy())
            y_test.append(labels.cpu().numpy())
            for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                test_hist["test_acc"].append((predicted == labels).sum().item())

    data_total = list(labels.view(-1).cpu().numpy())
    pred_data_total = list(predicted.view(-1).cpu().numpy())
    y_pred_list = [a.squeeze().tolist() for a in y_score][0]
    conf = confusion_matrix.tolist()
    total_acc = 100 * correct / total
    RB = 100 * class_correct[0] / class_total[0]
    NRB = 100 * class_correct[1] / class_total[1]
    miscore = f1_score(data_total, pred_data_total, average='micro')
    mascore = f1_score(data_total, pred_data_total, average='macro')
    ba = balanced_accuracy_score(data_total, pred_data_total)
    er = (confusion_matrix[0,1]+confusion_matrix[1,0])/sum(sum(confusion_matrix))
#     print()
#     print('[Test]')
#     print('- Total Accuracy : %d %%' % (100 * correct / total))            
#     print('- Accuracy of RB : %2d %%' % (RB))
#     print('- Accuracy of NRB : %2d %%' % (NRB))
#     print('- F-1 Micro Score : %.2f' % (float(miscore)))
#     print('- F-1 Macro Score : %.2f' % (float(mascore)))
    return conf, total_acc, RB, NRB, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er


def experiment(model, train_loader, test_loader, device, args):
    time_start = time.time()
    
    optimizer = optim.Adam(model.parameters(),lr=args.lr)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=args.step_size,
                                          gamma=args.gamma)
    
    list_train_loss = list()
    list_train_acc = list()
#     print('[Train]')
    for epoch in range(args.epoch):
        scheduler.step()
#         print('- Epoch :', epoch+1)
        model, train_loss, train_acc = train(model, device, optimizer, train_loader, criterion, args)
        list_train_loss.append(train_loss)
        list_train_acc.append(train_acc)
    
    conf, total_acc, RB, NRB, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er = test(model, device, test_loader, args)
    
    time_end = time.time()
    time_required = time_end - time_start
    
    args.list_train_loss = list_train_loss
    args.list_train_acc = list_train_acc
    args.data_total = data_total
    args.pred_data_total = pred_data_total
    args.conf = conf
    args.total_acc = total_acc
    args.RB = RB
    args.NRB = NRB
    args.miscore = miscore
    args.mascore = mascore
    args.time_required = time_required
    args.y_pred_list = y_pred_list
    args.ba = ba
    args.er = er
    
    save_checkpoint(epoch, model, optimizer, 'model.pt')
    
    return args


def experiment_test(model, test_loader, device, args):
    time_start = time.time()
    
    conf, total_acc, RB, NRB, miscore, mascore, data_total, pred_data_total, y_pred_list, ba, er = test(model, device, test_loader, args)
    
    time_end = time.time()
    time_required = time_end - time_start

    args.data_total = data_total
    args.pred_data_total = pred_data_total
    args.conf = conf
    args.total_acc = total_acc
    args.RB = RB
    args.NRB = NRB
    args.miscore = miscore
    args.mascore = mascore
    args.time_required = time_required
    args.y_pred_list = y_pred_list
    args.ba = ba
    args.er = er
        
    return args

In [ ]:
class GCNlayer(nn.Module):
    
    def __init__(self, n_features, conv_dim1, conv_dim2, conv_dim3, concat_dim, dropout):
        super(GCNlayer, self).__init__()
        self.n_features = n_features
        self.conv_dim1 = conv_dim1
        self.conv_dim2 = conv_dim2
        self.conv_dim3 = conv_dim3
        self.concat_dim =  concat_dim
        self.dropout = dropout
        
        self.conv1 = GCNConv(self.n_features, self.conv_dim1)
        self.bn1 = BatchNorm1d(self.conv_dim1)
        self.conv2 = GCNConv(self.conv_dim1, self.conv_dim2)
        self.bn2 = BatchNorm1d(self.conv_dim2)
        self.conv3 = GCNConv(self.conv_dim2, self.conv_dim3)
        self.bn3 = BatchNorm1d(self.conv_dim3)
        self.conv4 = GCNConv(self.conv_dim3, self.concat_dim)
        self.bn4 = BatchNorm1d(self.concat_dim)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = global_add_pool(x, data.batch)
        x = F.dropout(x, p=self.dropout, training=self.training)
        return x
    
class FClayer(nn.Module):
    
    def __init__(self, concat_dim, pred_dim1, pred_dim2, out_dim, dropout):
        super(FClayer, self).__init__()
        self.concat_dim = concat_dim
        self.pred_dim1 = pred_dim1
        self.pred_dim2 = pred_dim2
        self.out_dim = out_dim
        self.dropout = dropout

        self.fc1 = Linear(self.concat_dim, self.pred_dim1)
        self.bn1 = BatchNorm1d(self.pred_dim1)
        self.fc2 = Linear(self.pred_dim1, self.pred_dim2)
        self.fc3 = Linear(self.pred_dim2, self.out_dim)
    
    def forward(self, data):
        x = F.relu(self.fc1(data))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc3(x)
        return x
    
class Net(nn.Module):
    def __init__(self, args):
        super(Net, self).__init__()
        self.conv = GCNlayer(args.n_features, 
                              args.conv_dim1, 
                              args.conv_dim2, 
                              args.conv_dim3, 
                              args.concat_dim, 
                              args.dropout)

        self.fc = FClayer(args.concat_dim, 
                          args.pred_dim1, 
                          args.pred_dim2, 
                          args.out_dim, 
                          args.dropout)
        
    def forward(self, data):
        x = self.conv(data)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
args.epoch = 400
args.lr = 0.0001
args.optim = 'Adam'
args.step_size = 10
args.gamma = 0.9
args.dropout = 0.1
args.n_features = 75
dim = 512
args.conv_dim1 = dim
args.conv_dim2 = dim
args.conv_dim3 = dim
args.concat_dim = dim
args.pred_dim1 = dim
args.pred_dim2 = dim
args.out_dim = 2

BA = []
Sn = []
Sp = []
ER = []
for t in tqdm([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]):
    for r in tqdm(range(0, 1000, 10)):

        np.random.seed(args.seed)
        torch.manual_seed(args.seed)

        df = pd.read_csv('data/All-Public dataset.csv', low_memory=False)
        df = pd.concat([df['SMILES'], df['Class']], axis=1)

        X_train, X_test = train_test_split(df, test_size=t, shuffle=True, stratify=df['Class'], random_state=r)
        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)

        train_mols = make_mol(X_train)
        test_mols = make_mol(X_test)

        train_X = make_vec(train_mols)
        test_X = make_vec(test_mols)

        model = Net(args)
        model = model.to(device)

        train_loader = DataLoader(train_X, batch_size=len(train_X), shuffle=True, drop_last=True)
        test_loader = DataLoader(test_X, batch_size=len(test_X), shuffle=False, drop_last=True)

        dict_result = dict()
        args.exp_name = 'Test'
        result = vars(experiment(model, train_loader, test_loader, device, args))
        dict_result[args.exp_name] = copy.deepcopy(result)
        torch.cuda.empty_cache()

        res_df = pd.DataFrame(dict_result).transpose()
        c = res_df['conf'].iloc[0]
        confusion_matrix = np.array([c[0], c[1]], dtype=float)
        cm = confusion_matrix
        sensitivity1 = cm[1,1]/(cm[1,1]+cm[1,0])
        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
        er = (cm[0,1]+cm[1,0])/sum(sum(cm))
        BA.append(res_df['ba'].iloc[0])
        Sn.append(sensitivity1)
        Sp.append(specificity1)
        ER.append(er)
        
results_df = pd.DataFrame({'GCN_BA':BA, 'GCN_Sn':Sn, 'GCN_Sp':Sp, 'GCN_ER':ER})
results_df.to_csv('GCN_test_4_3.csv', index=False)
results_df